## Projeto de Modelo de Machine Learning para Detecção de Fraude

Para a criação desse modelo, serão utilizados dados da base de dados do Kaggle conhecida como **Fraud Detection Example**. Essa base de dados é uma fração de dados do **PaySim**, um simulador de dados financeiros muito usado para projetos voltados para detecção de fraude.

### Significado das Colunas

**step** - É uma unidade de tempo no mundo real. A cada "step" ou passo, significa 1 hora do tempo real. Ao todo, foram simulados 30 dias.

**type** - CASH-IN, CASH-OUT, DEBIT, PAYMENT e TRANSFER.

**amount** - Valor da transação em moeda local.

**nameOrig** - Cliente que iniciou a transação.

**oldbalanceOrg** - Saldo inicial antes da transação.

**newbalanceOrig** - Saldo após a transação.

**nameDest** - Cliente destinatário da transação.

**oldbalanceDest** - Destinatário do saldo inicial antes da transação. Note que não há informações para clientes que começam com M (comerciantes).

**newbalanceDest** - Novo destinatário do saldo após a transação. Note que não há informações para clientes que começam com M (comerciantes).

**isFraud** - São transações feitas pelos agentes fraudulentos dentro da simulação. Nesse cenário simulado, o comportamento fraudulento dos agentes visa lucrar ao assumir o controle das contas dos clientes e tentar esvaziar os fundos transferindo para outra conta e depois realizando o saque do sistema.

**isFlaggedFraud** - O modelo de negócios visa controlar transferências massivas de uma conta para outra e sinaliza tentativas ilegais. Uma tentativa ilegal neste conjunto de dados é uma tentativa de transferir mais de 200.00 em uma única transação.

### Importações

In [2]:
import pandas as pd

### Análise/manipulação Inicial

In [3]:
df = pd.read_csv('fraud_dataset_example.csv')

In [8]:
df = df[['isFraud',
       'isFlaggedFraud','step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig',
       'nameDest', 'oldbalanceDest', 'newbalanceDest']]

In [9]:
df.head()

,isFraud,isFlaggedFraud,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest
0,0,0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0
1,0,0,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0
2,1,0,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0
3,1,0,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0
4,0,0,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0


In [10]:
colunas = {
    'isFraud': 'fraude',
    'isFlaggedFraud': 'grande_fraude',
    'step': 'tempo', 
    'type': 'tipo', 
    'amount': 'valor', 
    'nameOrig': 'cliente_org', 
    'oldbalanceOrg': 'saldo_inicial_cliente_org', 
    'newbalanceOrig': 'saldo_novo_cliente_org',
    'nameDest': 'cliente_dest', 
    'oldbalanceDest': 'saldo_inicial_cliente_dest', 
    'newbalanceDest': 'saldo_novo_cliente_dest'
}

In [12]:
df = df.rename(columns = colunas)

In [16]:
df.describe().T #Sem NaN

,count,mean,std,min,25%,50%,75%,max
fraude,101613.0,1.141586e-03,3.376824e-02,0.00,0.00,0.00,0.00,1.00
grande_fraude,101613.0,0.000000e+00,0.000000e+00,0.00,0.00,0.00,0.00,0.00
tempo,101613.0,8.523457e+00,1.820681e+00,1.00,8.00,9.00,10.00,10.00
valor,101613.0,1.740901e+05,3.450199e+05,0.32,10016.59,53385.41,212498.40,10000000.00
saldo_inicial_cliente_org,101613.0,9.071753e+05,2.829575e+06,0.00,0.00,20190.47,194715.00,38939424.03
saldo_novo_cliente_org,101613.0,9.234992e+05,2.867319e+06,0.00,0.00,0.00,219217.76,38946233.02
saldo_inicial_cliente_dest,101613.0,8.810428e+05,2.399949e+06,0.00,0.00,21058.00,591921.70,34008736.98
saldo_novo_cliente_dest,101613.0,1.183998e+06,2.797761e+06,0.00,0.00,51783.43,1063121.64,38946233.02


Verificando a variável explicativa

In [19]:
df.groupby('fraude').tempo.count()

fraude
0    101497
1       116
Name: tempo, dtype: int64